In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset 
import torch
import torch.nn as nn

c:\Users\Timii\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd_xela_allfiles = pd.read_csv('data.csv', index_col=0)
pd_sliplabel_allfiles = pd.read_csv('labels.csv', index_col=0)

data = pd_xela_allfiles.to_numpy()
labels = pd_sliplabel_allfiles.to_numpy()

In [3]:
sc = MinMaxScaler()
sc.fit(data)
data = sc.transform(data)

In [4]:
data_prime = []
for t in range(data.shape[0]-9):
    data_prime.append(data[t:t+10])
data_prime = np.array(data_prime)
label_prime = labels[9:]
data_prime_v = data_prime[:]
label_prime = label_prime[:]

In [5]:
class Batch_Taxels(Dataset):
    
    def __init__(self, data_prime, label_prime, data_prime_v, label_prime_v, valid = None):
        self.x = data_prime
        self.y = label_prime
        self.xvalid = data_prime
        self.yvalid = label_prime
        self.valid = valid

    def __len__(self):
        if self.valid == True:
            return self.xvalid.shape[0]
        else:
            return self.x.shape[0]

    def __getitem__(self, idx):

        if self.valid == True:
            return self.xvalid[idx], self.yvalid[idx]
        else:
            return self.x[idx], self.y[idx]

#Divide the data into batches
dataset = Batch_Taxels(data_prime, label_prime, data_prime, label_prime)
dataset2 = Batch_Taxels(data_prime, label_prime, data_prime, label_prime, valid = True)

xelaloader = DataLoader(dataset = dataset, batch_size=32, shuffle=True)
xelaloadervalid = DataLoader(dataset = dataset2, batch_size=32, shuffle=True)

In [6]:

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
        
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size) 
        c0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        
        # Forward propagate LSTM
        out, (hn, cn) = self.lstm(x, [h0, c0])  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [7]:

# Define the parameters of the model
input_size = 48
hidden_size = 32
num_layers = 1
num_classes = 1
num_epochs = 1

# Instantiate the model
model = LSTM(input_size, hidden_size, num_layers, num_classes)

loss = nn.BCEWithLogitsLoss()#pos_weight=torch.tensor([2.0]))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_loss = []
valid_loss = []

t_loss = []
v_loss = []

t_acc = []
v_acc = []

t_acc_t = []
v_acc_t = []

for epoch in range(num_epochs):
    #Train per batch
    
    total = 0
    correct = 0
    l_xelaloader = 0

    model.train()
    for x, y in (xelaloader):

        #Forward pass
        y_pred = model(x.float())
       
        #compute the loss
        l = loss(y_pred, y.float())

        #empty the gradients
        optimizer.zero_grad()

        #compute the gradient
        l.backward()

        #update the weights
        optimizer.step()


        #append each loss per batch
        train_loss.append(l.item())

        #accuracy
        total += y.size(0)
        correct += y_pred.round().eq(y).sum().item()
        l_xelaloader += x.shape[0]
        
    
    t_acc = correct/l_xelaloader
    t_acc_t.append(t_acc)

    total = 0
    correct = 0
    l_xelaloader = 0

    #calculate and plot the validation loss
    model.eval()
    for (x,y) in (xelaloadervalid):
        y_pred_test = model(x.float())
        lv = loss(y_pred_test, y)
        #append the loss per batch
        valid_loss.append(lv.item())

        #accuracy
        total += y.size(0)
        correct += y_pred_test.round().eq(y).sum().item()
        l_xelaloader += x.shape[0]
        
    v_acc = correct/l_xelaloader
    v_acc_t.append(v_acc)

    #append the total loss and accuracy per epoch
    t_loss.append(np.mean(train_loss))
    v_loss.append(np.mean(valid_loss))

    print(f'For training epoch {epoch+1}, loss ={l:.8f}', f'For validation epoch {epoch+1}, loss ={lv:.8f}'  )

For training epoch 1, loss =0.05744501 For validation epoch 1, loss =0.02551020
